### Load traning data and do one-hot encoding on labels

In [1]:
import csv

# every 30 lines is a different speaker. Each recording is decomposed into 9 arrays of 4 points (9 lines from the original plot)
with open('training_after_PCA_after_taking_4.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile))

# generate labels for the training 
integer_encoded = []
for i in range(9):
    integer_encoded += [i] * 30

onehot_encoded = []
for elem in integer_encoded:
    letter = [0 for i in range(9)]
    letter[elem] = 1
    onehot_encoded.append(letter)

y_train=onehot_encoded
x_train=data

### Load testing data and do one-hot encoding on labels

In [2]:
import csv

# every 30 lines is a different speaker. Each recording is decomposed into 9 arrays of 4 points (9 lines from the original plot)
with open('testing_after_PCA_after_taking_4.csv', newline='') as csvfile:
    data_test = list(csv.reader(csvfile))


# generate labels for the training 
integer_encoded = [0]*31 + [1]*35 + [2]*88 + [3]*44 + [4]*29 + [5]*24 + [6]*40 + [7]*50 + [8]*29

onehot_encoded = []
for elem in integer_encoded:
    letter = [0 for i in range(9)]
    letter[elem] = 1
    onehot_encoded.append(letter)

y_test=onehot_encoded
x_test=data_test
print(len(y_test))
print(len(x_test))

370
370


### Convert data from str to list of ints

In [3]:
import string

def Convert_to_list_of_ints(data): 
    t=[]
    c=[]
    for elem in data:
        for i in range(9):
            temp = elem[i].strip(string.punctuation)
            temp = temp.replace(" ", "")
            temp = temp.replace("'","")
            temp = list(map(float, temp.split(",")))
            c.append(temp) #concatenate the sub arrays for each of the 9 lines to make one big array
        t.append(c)
        c=[]
    return t

x_train=Convert_to_list_of_ints(x_train)
x_test=Convert_to_list_of_ints(x_test)

print(len(x_train))
print(len(x_test))


270
370


In [4]:
print(x_train[0])

[[1.7693870767093212, -0.8460164272960151, 0.5534737237068844, 1.8703358874300287], [1.8133210106900062, -2.3602846275855365, -1.98182238750905, -0.4721725842355035], [0.40750881492947255, 0.8992262468033351, 1.909293447861917, 0.7356027461878615], [1.9100375843927693, 1.045651158944131, 0.5428003387806436, 0.1371033075460577], [1.9251124748613964, 1.634097183138999, 0.5103132343254394, 2.1276938207885956], [1.0018328960752336, 0.8209451490134135, 0.49295765921184925, -0.8600332985100363], [0.13362468773147332, 0.055888058958342354, -0.2736495305870298, -0.3932043731639428], [0.28255814116254524, -0.3760625100493803, -0.02034860759700484, -0.351702426845183], [0.7378421207748456, 0.4384662613411774, 0.3049287052925835, 1.3331808535790062]]


In [12]:
import numpy as np
import random

# shuffle training data
c = list(zip(x_train, y_train))
random.shuffle(c)
x_train, y_train = zip(*c)

## Turn the lists into np.arrays
x_train, y_train = np.array(x_train), np.array(y_train)


## we also have to convert our data into three-dimensional format
x_train = np.reshape(x_train, (x_train.shape[0], 4,9))

In [13]:
# shuffle testing data
b = list(zip(x_test, y_test))
random.shuffle(b)
x_test, y_test = zip(*b)

## Turn the lists into np.arrays
x_test, y_test = np.array(x_test), np.array(y_test)

## we also have to convert our data into three-dimensional format
x_test = np.reshape(x_test, (x_test.shape[0], 4,9))

### The model

In [22]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

model = Sequential()
model.add(LSTM(units=50,activation='relu', recurrent_activation='sigmoid', return_sequences=True,input_shape=(x_train.shape[1], 9),dropout=0.2,recurrent_dropout=0.2))
model.add(LSTM(units=50,activation='relu', recurrent_activation='sigmoid', return_sequences=True, dropout=0.2,recurrent_dropout=0.2))
model.add(LSTM(units=50, activation='relu', recurrent_activation='sigmoid',return_sequences=False, dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(units=9,activation='softmax'))
model.compile(optimizer = 'adam', loss = 'mean_squared_error',metrics = ['accuracy'])
history = model.fit(x_train, y_train, batch_size=5, epochs=30)

# saving the model
model.save('keras_ML1.h5')

Epoch 1/30
270/270 [==============================] - 2s 6ms/step - loss: 0.0987 - accuracy: 0.2074
Epoch 2/30
270/270 [==============================] - 0s 1ms/step - loss: 0.0965 - accuracy: 0.2185
Epoch 3/30
270/270 [==============================] - 0s 2ms/step - loss: 0.0848 - accuracy: 0.3444
Epoch 4/30
270/270 [==============================] - 0s 1ms/step - loss: 0.0745 - accuracy: 0.4444
Epoch 5/30
270/270 [==============================] - 0s 1ms/step - loss: 0.0681 - accuracy: 0.4815
Epoch 6/30
270/270 [==============================] - 0s 1ms/step - loss: 0.0644 - accuracy: 0.5556
Epoch 7/30
270/270 [==============================] - 0s 1ms/step - loss: 0.0604 - accuracy: 0.5704
Epoch 8/30
270/270 [==============================] - 0s 1ms/step - loss: 0.0614 - accuracy: 0.5741
Epoch 9/30
270/270 [==============================] - 0s 1ms/step - loss: 0.0569 - accuracy: 0.5926
Epoch 10/30
270/270 [==============================] - 0s 1ms/step - loss: 0.0550 - accuracy: 0.6444

In [23]:
from keras.models import load_model

# testing the model
model = load_model('keras_ML1.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.04957976607052056
Test accuracy: 0.6918919086456299
